<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/%EC%9D%B4%EB%AF%B8%EC%A7%80%EC%B2%98%EB%A6%AC(with_Tensorflow).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN과 특성 개념  
 - 핵심 특징을 올바르게 추출
 - 다층 신경망과 다층 합성곱 신경망은 각 층별로 저수준 특성을 연결하여, 고수준 특성을 만듦.
 - CNN은 입력 이미지에서 feature map을 만든다.
 - CNN이 이미지 관련 작업을 잘 수행하는 이유는 다음과 같은 2개의 아이디어 때문이다.
  - 희소 연결 : 특성 맵에 있는 하나의 원소는 작은 픽셀 패치 하나에만 연결된다.
  - 파라미터 공유 : 동일한 가중치가 입력 이미지의 모든 패치에 사용된다.  
  즉, 이 두 아이디어의 결과로 네트워크의 가중치(파라미터) 개수가 극적으로 감수하고, 중요 특징을 잡아내는 능력이 향상된다.

## 1D 이산 합성곱 수행
이산 합성곱은 CNN의 기본 연산이다. 텐서플로우 같은 패키지의 실제 합성곱 연산은 훨씬 효율적으로 구성되어 있다.

In [ ]:
import numpy as np
def conv1d(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad,
                                   x_padded,
                                   zero_pad])
    res = []
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
        
    return np.array(res)

In [ ]:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print("Conv1d 구현 :", conv1d(x, w, p=2, s=1))

Conv1d 구현 : [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [ ]:
print("numpy 결과 :", np.convolve(x, w, mode='same'))

numpy 결과 : [ 5 14 16 26 24 34 19 22]


## 2D 합성곱 출력 크기 계산  
합성곱 출력 크기는 입력 벡터 위를 필터 w가 이동하는 전체 횟수로 결정된다. 

In [ ]:
import numpy as np
import scipy.signal

In [ ]:
def conv2d(X, W, p=(0, 0), s=(1, 1)):
    W_rot = np.array(W)[::-1, ::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0] + X_orig.shape[0],
             p[1]:p[1] + X_orig.shape[1]] = X_orig

    res_ = []
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0]) /s[0]) + 1, s[0]):
        res_.append([])
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1]) / s[1]) + 1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0],
                             j:j+W_rot.shape[1]]
            res_[-1].append(np.sum(X_sub * W_rot))
    
    return (np.array(res_))

In [ ]:
X = [[1, 3, 2, 4], [5, 6, 1, 3], [1, 2, 0, 2], [3, 4, 3, 2]]
W = [[1, 0, 3], [1, 2, 1], [0, 1, 1]]

print("Conv2D 구현:\n", conv2d(X, W, p=(1,1), s=(1, 1)))

Conv2D 구현:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [ ]:
print("사이파이 결과:\n", scipy.signal.convolve2d(X, W, mode='same'))

사이파이 결과:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


## Tensorflow를 활용한 심층 합성곱 신경망 구현

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train.shape, y_train.shape

11493376/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,))

In [3]:
from tensorflow.keras import layers

In [4]:
model = tf.keras.Sequential([
                             layers.Conv2D(64, (3,3), activation='relu'),
                             layers.MaxPooling2D((2,2)),
                             layers.Conv2D(128, (3,3), activation='relu'),
                             layers.MaxPooling2D((2,2)),
                             layers.Conv2D(256, (3,3), activation='relu'),
                             layers.MaxPooling2D((2,2)),
                             layers.Flatten(),
                             layers.Dense(32, activation='relu'),
                             layers.Dense(10, activation='softmax')
])

In [5]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [6]:
X_train_, X_test_ = X_train[..., tf.newaxis], X_test[..., tf.newaxis]

In [7]:
X_train_.shape

(60000, 28, 28, 1)

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics='accuracy')

In [9]:
X_train_, X_test_ = X_train_/255, X_test_/255

In [10]:
model.fit(X_train_, y_train, epochs=3, validation_split=0.2)

Epoch 1/3
1500/1500 [==============================] - 82s 55ms/step - loss: 0.1986 - accuracy: 0.9388 - val_loss: 0.0734 - val_accuracy: 0.9791
Epoch 2/3
1500/1500 [==============================] - 80s 53ms/step - loss: 0.0645 - accuracy: 0.9805 - val_loss: 0.0594 - val_accuracy: 0.9829
Epoch 3/3
1500/1500 [==============================] - 81s 54ms/step - loss: 0.0437 - accuracy: 0.9865 - val_loss: 0.0761 - val_accuracy: 0.9778


In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (32, 26, 26, 64)          640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 13, 13, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 11, 11, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 5, 5, 128)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 3, 3, 256)           295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (32, 1, 1, 256)           0         
_________________________________________________________________
flatten (Flatten)            (32, 256)                 0

In [18]:
model.weights[2].shape

TensorShape([3, 3, 64, 128])